In [2]:
#import libraries
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium # map rendering library

print('Libraries imported')

Libraries imported


In [3]:
# Scrape Wikipedia page to obtain postal codes in Toronto
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [32]:
# Let's take a quick look at the data
#print(soup.prettify())
table = soup.find('table','wikitable sortable')
#print(soup.table.prettify())

In [45]:
# Transform postal code data into a pandas dataframe
column_names = ['Postcode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns = column_names)

# loop through to find postcode, borough, neighborhood 
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
#           neighborhood = td.text
    toronto_data = toronto_data.append({'Postcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
#print(toronto_data)

In [54]:
# clean dataframe 
toronto_data = toronto_data[toronto_data.Borough!='Not assigned']
toronto_data = toronto_data[toronto_data.Borough!= 0]
toronto_data.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto_data.shape[0]):
    if toronto_data.iloc[i][2] == 'Not assigned':
        toronto_data.iloc[i][2] = toronto_data.iloc[i][1]
        i = i+1
                                 
df = toronto_data.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
#df.head(10)
#print(df.shape)

In [53]:
# remove not assigned borough
df = df.drop(df[(df.Borough == "Not assigned")].index)
#df.head(10)

In [59]:
# combine neighborhoods with the same postal code
def grouped_neighborhood(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postcode', 'Borough'])
df2 = grp.apply(grouped_neighborhood).reset_index(name='Neighborhood')

In [62]:
# copy borough value to a not assigned neighborhood
df.Neighborhood.replace("Not assigned", df.Borough, inplace = True)
# df.head(10)

In [61]:
print(df2.shape)
df2.head()

(103, 3)


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
